## DRS document
Defining the functions first and then applying that on a selection of suitable document types (ACs and Orders like).

In [2]:
import PyPDF2
import re
import pandas as pd

def get_bookmarks(outlines, pdf, parent_position_in_bookmark_tree):

    def get_bookmark_content(outline, pdf):
        if pdf.get_destination_page_number(outline)>=0:
            try:
                repattern_title_number_ac = 'AC 2[13759]'+\
                                            '('+\
                                                '\.[0-9]{1,5}[A-Z]?\.?[\s]{1,3}§'+\
                                                    '|'+\
                                                '(?= MG [0-9]{1,2}\.?)'+\
                                            ')' 
                repattern_title_text_ac = '('+\
                                            '2[13759]\.[0-9]{1,5}\.?'+\
                                            ' (\(A[A-Za-z 0-9-]*\))?' +\
                                                    '|'+\
                                            'MG [0-9]{1,2}\.?'+\
                                            ')'      
                                                                                                                         
                repattern_title_prefix = '(VOLUME|volume|Volume|Section|SECTION|section|Section|PART|SUBPART|CHAPTER|chapter|Chapter|APPENDIX|Appendix|appendix)+[\s]{1,2}'
                repattern_greek = '[ivx]{2,3}|[IVX]{2,3}'
                repattern_title_prevent_firstletter_mismatch = '(?!([A-Z]{2,10}|[a-z]{2,10}))'    
                repattern_title_number_basic = '[\(]?('+repattern_greek+'|[1-9][0-9]{0,3}|[A-Za-z])'+repattern_title_prevent_firstletter_mismatch
                repattern_title_delimiter = '(-|\)|\.)'

                repattern_title_number ='('+\
                                            repattern_title_number_ac+\
                                        '|'+\
                                            '('+repattern_title_prefix+')?'+\
                                            '('+\
                                                repattern_title_delimiter.join([repattern_title_number_basic, 
                                                                                repattern_title_number_basic, 
                                                                                '[ ]?'+repattern_title_number_basic, 
                                                                                repattern_title_number_basic])+\
                                            '|'+\
                                                repattern_title_delimiter.join([repattern_title_number_basic, 
                                                                                repattern_title_number_basic, 
                                                                                '[ ]?'+repattern_title_number_basic])+\
                                            '|'+\
                                                repattern_title_delimiter.join([repattern_title_number_basic, 
                                                                                repattern_title_number_basic])+\
                                            '|'+\
                                                repattern_title_delimiter.join([repattern_title_number_basic])+\
                                            ')'+\
                                        ')'+\
                                        '[\s]{0,2}'+repattern_title_delimiter+'?'+'[\s]{0,2}'
                
                repattern_before_title = '(?<=( |\n))'
                
                for offset in [0,-1,-2,1,2]:                
                    line_with_title = None
                    nb_keywords = 5
                    while line_with_title is None and nb_keywords>0:
                        outline_title_cleaned = re.sub('[^a-zA-Z ,/\d\s\.:-]', '.', outline.title)

                        title_shortened_variable_spaces = re.sub('[ ]+', ' ', outline_title_cleaned).split(' ')[:nb_keywords]
                        title_shortened_variable_spaces = title_shortened_variable_spaces[:2]+\
                                                        [keyword[:2]+'-?'.join(keyword[2:]) for keyword in title_shortened_variable_spaces[2:]]
                        title_shortened_variable_spaces = '[ ]*'.join(title_shortened_variable_spaces)

                        repattern_title_line = '('+repattern_before_title+repattern_title_number+')?'+\
                                                title_shortened_variable_spaces+\
                                                '.*(\n|\r|$)'
                        try:
                            line_with_title = re.search(repattern_title_line,
                                                        pdf.pages[max(0,pdf.get_destination_page_number(outline)+offset)].extract_text())
                        except:
                            pass                        

                        if line_with_title is not None:
                            break
                        else:
                            nb_keywords = nb_keywords - 1

                    if line_with_title is not None:
                        break
                    
                sentence_around_title = pdf.pages[max(0,
                                                      pdf.get_destination_page_number(outline)+offset)].extract_text()[line_with_title.span()[0]:\
                                                                                                                        max(line_with_title.span()[1],\
                                                                                                                            line_with_title.span()[0]+len(outline.title))]
                title_number = re.search(repattern_title_number, sentence_around_title)

                title_max_length = 200
                title_text_end_delimiter = '(\.|\:|\?|\n|\r|$)?'
                repattern_title_text = '('+repattern_title_text_ac+')?'+\
                                        '[A-Za-z,\/\-\s0-1\(\)\[\]]'+\
                                        '{3,'+str(title_max_length)+'}'+title_text_end_delimiter# [\s]{1,3}
                if title_number is not None:
                    title_text = re.search(repattern_title_text, 
                                           sentence_around_title[len(title_number.group(0)):])
                    string_after_title = title_text.string[title_text.span()[1]:
                                                            (title_text.span()[1]+30)]

                    bookmark_content = [outline.title,
                                        str(pdf.get_destination_page_number(outline)),
                                        str(pdf.get_destination_page_number(outline)+offset),
                                        title_number.group(0),
                                        title_text.group(0),
                                        string_after_title,
                                        str(nb_keywords)
                                        ]
                    pass
                else:
                    title_text = re.search(repattern_title_text, sentence_around_title[len(title_number.group(0)):])
                    string_after_title = title_text.string[title_text.span()[1]:
                                                           (title_text.span()[1]+30)]

                    bookmark_content = [outline.title,
                                        str(pdf.get_destination_page_number(outline)),
                                        str(pdf.get_destination_page_number(outline)+offset),
                                        '',
                                        title_text.group(0),
                                        string_after_title,
                                        str(nb_keywords)
                                        ]
                    pass
            except:
                bookmark_content = [outline.title,
                                    pdf.get_destination_page_number(outline),
                                    '',
                                    '',
                                    '',
                                    '',
                                    ''
                                    ]
                pass
        else:
            bookmark_content = [outline.title,
                                pdf.get_destination_page_number(outline),
                                '',
                                '',
                                '',
                                '',
                                ''
                                ]
            pass
        
        return bookmark_content


    headers = ['position in bookmark tree',
                'original pdf bookmark title (outline.title)',
                'original pdf bookmark page number',
                'corrected pdf bookmark page number',
                'extracted title number',
                'extracted title text',
                'first 20 characters after title',
                'nb_keywords for title search']

    df_content = pd.DataFrame(columns=headers,
                            dtype="string")
    
    n_section = -1

    for outline in outlines:
        if type(outline) == PyPDF2.generic._data_structures.Destination:
            n_section = n_section + 1
            if parent_position_in_bookmark_tree=='':
                current_position_in_bookmark_tree = str(n_section)
            else:
                current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'>'+str(n_section)

            df_thisbookmark_content = get_bookmark_content(outline, pdf)
            df_thisbookmark_content = [current_position_in_bookmark_tree]+df_thisbookmark_content

            try:
                df_thisbookmark_content = pd.DataFrame(dict(zip(headers, df_thisbookmark_content)),
                                                    dtype="string", 
                                                    index=[0])
            except UnicodeDecodeError:
                df_thisbookmark_content[1] = df_thisbookmark_content[1].decode('utf-8','ignore')+' (Python Exception: title UnicodeDecodeError)'
                df_thisbookmark_content = pd.DataFrame(dict(zip(headers, df_thisbookmark_content)),
                                                    dtype="string", 
                                                    index=[0])

            df_content = pd.concat([df_content, 
                                    df_thisbookmark_content], 
                                    ignore_index=True)
        elif type(outline) == list:
            if parent_position_in_bookmark_tree=='':
                current_position_in_bookmark_tree = str(n_section)
            else:
                current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'>'+str(n_section)

            extracted_content = get_bookmarks(outline, pdf, current_position_in_bookmark_tree)

            df_content = pd.concat([df_content, 
                                    extracted_content], 
                                    ignore_index=True)
        else:
            raise Exception
        
    return df_content

# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__083AD53732DC0D4686258347005037DF.pdf") # AC 43-13
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__99C827DB9BAAC81B86256B4500596C4E.pdf") # AC 43-13
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__C7CCE9FCA6D7E34786257D41004C3E63.pdf") # AC 29
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\ORDER_8900.1\pdf\Current__DRSDOCID166392477020230614171412.pdf") # Order
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__C2774DBA5FFEE8D286258088005528A9.pdf") # Order


# import glob, os
# matching_file = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'raw data', '**','**','*5934B72DA528D2CC862585230041269E*'))
# drs_doc = PyPDF2.PdfReader(matching_file[0])

# df_content = get_bookmarks(outlines = drs_doc.outline, pdf = drs_doc, parent_position_in_bookmark_tree='') # [8:]

In [3]:
import os
import glob
import pandas as pd
import PyPDF2
import datetime

drs_doc_types = ['AC',
                 'ORDER_8300.10',
                 'ORDER_8400.10',
                 'ORDER_8700.1',
                 'ORDER_8740.1',
                 'ORDER_8900.1',
                 'ORDERS']

df_all_content = None

for drs_doc_type in drs_doc_types:
    df_index = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'index', drs_doc_type+'*.parquet'))[-1]
    df_index = pd.read_parquet(df_index)

    for index, row in df_index.iterrows():
        matching_file = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'raw data', drs_doc_type, 'pdf','*'+row['documentGuid']+'*.pdf'))
        if len(matching_file)>0:
            if row['drs:status']=='Current':
                if 'drs:docID'  in row.index:
                    print(row['drs:docID']+': '+row['documentGuid'])
                elif 'drs:documentNumber' in row.index:
                    print(row['drs:documentNumber']+': '+row['documentGuid'])

                drs_doc = PyPDF2.PdfReader(matching_file[0])
                df_content = get_bookmarks(outlines = drs_doc.outline, 
                                           pdf = drs_doc, 
                                           parent_position_in_bookmark_tree='')

                if len(df_content)>0:                    
                    df_content.insert(loc=0, 
                                      column='documentGuid', 
                                      value=pd.Series(data=row['documentGuid'], index=df_content.index, dtype="string"))
                    if df_all_content is not None:
                        df_all_content = pd.concat([df_all_content, 
                                                    df_content], 
                                                    ignore_index=True)
                    else:
                        df_all_content = df_content
                    print('   -> '+str(df_all_content.shape))

                
df_all_content.to_parquet(os.path.join(os.getcwd(), 
                                       'data', 
                                       'DRS', 
                                       'extracted_text_for_'+'-'.join(drs_doc_types)+'-'+datetime.datetime.now().strftime("%Y%m%d - %H%M%S")+'.parquet'))

AC 21-43A: E7E9387A5386881C86257ED9005A1B5A


#### Debugging

In [1]:
import pandas as pd

# df_all_content = pd.read_parquet(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\extracted_text_for_AC-ORDER_8300.10-ORDER_8400.10-ORDER_8700.1-ORDER_8740.1-ORDER_8900.1-ORDERS.parquet")
df_all_content = pd.read_parquet(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\extracted_text_for_AC-ORDER_8300.10-ORDER_8400.10-ORDER_8700.1-ORDER_8740.1-ORDER_8900.1-ORDERS-20231214 - 020735.parquet")

## Non DRS document (AIM...)
Defining the functions first and then applying that on a selection of Non DRS documents.

In [1]:
import PyPDF2
import pandas as pd

# pdf.pages[pdf.get_destination_page_number(pdf.outline[n]):
#                             pdf.get_destination_page_number(pdf.outline[n+1])].extract_text()

def get_bookmarks(outlines, pdf, parent_position_in_bookmark_tree):
    headers = ['position in bookmark tree',
                'original pdf bookmark title (outline.title)',
                'original pdf bookmark page number'
                ]
    df_content = pd.DataFrame(columns=headers,
                            dtype="string")    
    n_section = -1

    for outline in outlines:
        if type(outline) == PyPDF2.generic._data_structures.Destination:
            n_section = n_section + 1
            if parent_position_in_bookmark_tree=='':
                current_position_in_bookmark_tree = str(n_section)
            else:
                current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'>'+str(n_section)

            bookmark_content = [current_position_in_bookmark_tree,
                                outline.title,
                                str(pdf.get_destination_page_number(outline))
                                ]                        
            df_thisbookmark_content = pd.DataFrame(dict(zip(headers, bookmark_content)),
                                                dtype="string", 
                                                index=[0])            
            df_content = pd.concat([df_content, 
                                    df_thisbookmark_content], 
                                    ignore_index=True)
                
        elif type(outline) == list:
            if parent_position_in_bookmark_tree=='':
                current_position_in_bookmark_tree = str(n_section)
            else:
                current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'>'+str(n_section)

            extracted_content = get_bookmarks(outline, pdf, current_position_in_bookmark_tree)

            df_content = pd.concat([df_content, 
                                    extracted_content], 
                                    ignore_index=True)
        
    return df_content

In [2]:
import os
import glob
import pandas as pd
import PyPDF2
import datetime

raw_documents = glob.glob(os.path.join(os.getcwd(), 'data', 'NotInDRS', 'raw data', 'NotInDRS', 'pdf', '*.pdf'))
df_all_content = None

for raw_document in raw_documents:
    drs_doc = PyPDF2.PdfReader(raw_document)
    df_content = get_bookmarks(outlines = drs_doc.outline, pdf = drs_doc, parent_position_in_bookmark_tree='')

    if len(df_content)>0:                    
        df_content.insert(loc=0, column='filename', value=os.path.basename(raw_document))
        if df_all_content is not None:
            df_all_content = pd.concat([df_all_content, 
                                        df_content], 
                                        ignore_index=True)
        else:
            df_all_content = df_content
        print('   -> '+str(df_all_content.shape))

                
df_all_content.to_parquet(os.path.join(os.getcwd(), 
                                       'data', 
                                       'NotInDRS', 
                                       'dataset',
                                       'extracted_text',
                                       'extracted_text_for_DocNotInDRS'+'-'+datetime.datetime.now().strftime("%Y%m%d - %H%M%S")+'.parquet'))

   -> (73, 4)
   -> (187, 4)
   -> (372, 4)
   -> (679, 4)
   -> (704, 4)


c:\Users\victor\anaconda3\envs\faa-nlp-drs\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Post-process to split AIP and AIM with one more section level

In [4]:
import pandas as pd
import glob
import os
from pypdf import PdfReader
import datetime
# import PyPDF2 import PdfReader
import re
import numpy as np

predatasets = sorted(glob.glob(os.path.join(os.getcwd(), 'data', 'NotInDRS', 'dataset', 'extracted_text', '*.parquet')), 
                    key=os.path.getctime, 
                    reverse=True)
predatasets = [val for val in predatasets if '(augmented)' not in val]
df_predataset = pd.read_parquet(predatasets[0])

filename_aim = 'Aeronautical Information Manual (AIM) Basic with Change 1.pdf'
filename_aip = 'Aeronautical Information Publication (AIP) Basic with Amendments 1, 2 and 3.pdf'

pdf_aim = PdfReader(os.path.join(os.getcwd(), 'data', 'NotInDRS', 'raw data', 'NotInDRS', 'pdf', filename_aim))
pdf_aip = PdfReader(os.path.join(os.getcwd(), 'data', 'NotInDRS', 'raw data', 'NotInDRS', 'pdf', filename_aip))

re_prefix = '[ \s\r\n]'
re_sufix = '[\r\n]'
re_pattern_aim = re.compile(re_prefix+
                            '[0-9]{1,2}[−-][0-9]{1,2}[−-][0-9]{1,2}\. [A-Za-z \(\)\-−\/]{6,80}'+
                            re_sufix)
re_pattern_aip1 = re.compile(re_prefix+
                             '[0-9]{1,2}\. [A-Za-z \(\)\-−\/]{6,80}'+
                             re_sufix)

list_split4nonDRS = [[filename_aim, pdf_aim, re_pattern_aim],
                     [filename_aip, pdf_aip, re_pattern_aip1]]

for settings in list_split4nonDRS:
    filename, pdf, re_pattern = settings
    df_tobeadded = pd.DataFrame(columns=['previous_index']+list(df_predataset),
                                dtype="string")
    for n in range(0, len(pdf.pages)):
        pattern_findings = re_pattern.findall(pdf.pages[n].extract_text())
        if len(pattern_findings)>0:
            for pattern_finding in pattern_findings:
                first_letter_lower_case = [w[0].islower() for w in pattern_finding.split(' ') if (len(w)>3 and w[0] not in '0123456789')][1:]
                if np.sum(first_letter_lower_case)<2:
                    previous_index = df_predataset[(df_predataset["filename"]==filename) & 
                                                (df_predataset["original pdf bookmark page number"].apply(int)<=n) &
                                                (df_predataset["original pdf bookmark page number"].apply(int)>=0)].index[-1]
                    df_new_content = pd.DataFrame(dict(zip(['previous_index']+list(df_predataset), 
                                                            [previous_index,
                                                            filename,
                                                            df_predataset.loc[previous_index,"position in bookmark tree"]+'>0',
                                                            pattern_finding[1:-1],
                                                            str(n)])),
                                                        dtype="string", 
                                                        index=[0])            
                    df_tobeadded = pd.concat([df_tobeadded, 
                                            df_new_content], 
                                            ignore_index=True)

    counter_next_section = 0

    df_tobeadded = df_tobeadded.iloc[::-1]
    for idx, row in df_tobeadded.iterrows():
        df_predataset = pd.concat([df_predataset.iloc[:int(row['previous_index'])+1+counter_next_section], 
                                pd.DataFrame(dict(zip(list(df_predataset), row[list(df_predataset)])),
                                                dtype="string",
                                                index=[0]), 
                                    df_predataset.iloc[int(row['previous_index'])+1+counter_next_section:]]).reset_index(drop=True)

df_predataset.to_parquet(os.path.join(os.getcwd(), 
                                    'data', 
                                    'NotInDRS', 
                                    'dataset',
                                    'extracted_text',
                                    'extracted_text (augmented)_for_DocNotInDRS'+'-'+datetime.datetime.now().strftime("%Y%m%d - %H%M%S")+'.parquet'))

KeyboardInterrupt: 